In [205]:
#importing library
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV

In [206]:
#loading the dataset
df=pd.read_csv('../input/insurance/insurance.csv')
#showing the dataset
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [207]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [208]:
#checking for missing values in the dataset
print("Total Missing Value:",df.isna().sum().sum())

Total Missing Value: 0


In [209]:
df.select_dtypes('object').columns

Index(['sex', 'smoker', 'region'], dtype='object')

In [210]:
#creating dictionary to get the list of unique value in each column
{column:list(df[column].unique()) for column in df.select_dtypes('object').columns}

{'sex': ['female', 'male'],
 'smoker': ['yes', 'no'],
 'region': ['southwest', 'southeast', 'northwest', 'northeast']}

In [211]:
#creating binary_encode
def binary_encode(df,column,positive_value):
    #copying the dataframe
    df=df.copy()
    #applying lambda function to convert binary column into zero and one
    df[column]=df[column].apply(lambda x:1 if x==positive_value else 0)
    #returning the dataframe
    return df

In [212]:
def onehot_encode(df,column,prefix):
    #copying the dataframe
    df=df.copy()
    #using dummies function 
    dummies=pd.get_dummies(df[column],prefix=prefix)
    #concating the original dataframe to dummies column
    df=pd.concat([df,dummies],axis=1)
    #dropping the original dataframe
    df=df.drop(column,axis=1)
    #returning the dataframe
    return df
    
    

In [213]:
#preprocss_inputs
def preprocess_inputs(df,scaler,train_size=0.7):
    #binary encode sex and smoker columns
    #using binary_encode function 
    df=df.copy()
    df=binary_encode(df,'sex','male')
    #using binary_encode function 
    df=binary_encode(df,'smoker','yes')
    #using onehot_encode to encode the non binary object columns
    df=onehot_encode(df,'children','ch')
    df=onehot_encode(df,'region','re')
    #spliting the data in x and y
    #storing the charges column to y variable
    y=df['charges'].copy()
    #storing the features column to a x variable
    x=df.drop('charges',axis=1).copy()
    
    
    x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)
    #Split into train test split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=train_size,random_state=1)
    
    #returning the dataframe
    return x_train,x_test,y_train,y_test

In [214]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [215]:

#calling the model
x_train,x_test,y_train,y_test=preprocess_inputs(df,scaler=StandardScaler())

In [216]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(936, 14)
(402, 14)
(936,)
(402,)


In [217]:
x_train

,age,sex,bmi,smoker,ch_0,ch_1,ch_2,ch_3,ch_4,ch_5,re_northeast,re_northwest,re_southeast,re_southwest
744,0.768473,0.989591,-0.697746,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,1.765481,-0.611324,-0.566418
363,-1.296362,-1.010519,-0.699387,-0.507463,-0.866781,1.769076,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
10,-1.011557,0.989591,-0.728915,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,1.769076,-0.566418,-0.611324,-0.566418
970,0.768473,-1.010519,-0.410668,-0.507463,-0.866781,-0.565267,-0.467525,2.742680,-0.137987,-0.116775,-0.565267,-0.566418,1.635795,-0.566418
634,0.839674,0.989591,1.482405,-0.507463,-0.866781,1.769076,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,1.480485,0.989591,-0.289276,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
905,-0.940356,-1.010519,-0.214635,-0.507463,-0.866781,-0.565267,2.138925,-0.364607,-0.137987,-0.116775,1.769076,-0.566418,-0.611324,-0.566418
1096,0.839674,-1.010519,0.704834,1.970587,-0.866781,-0.565267,2.138925,-0.364607,-0.137987,-0.116775,1.769076,-0.566418,-0.611324,-0.566418
235,0.056461,-1.010519,-1.385093,1.970587,-0.866781,-0.565267,2.138925,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,1.635795,-0.566418


In [218]:
y_train

744      8827.20990
363      2597.77900
10       2721.32080
970     10702.64240
634      9391.34600
           ...     
715     12146.97100
905      4564.19145
1096    44641.19740
235     19444.26580
1061    11554.22360
Name: charges, Length: 936, dtype: float64

In [219]:
#Training the Model
models={
    'OLS Model': LinearRegression(),
    'L2 Model': Ridge(),
    'L1 Model': Lasso(),
    'ElasticNet Model': ElasticNet(),
    'L2 CV Model': RidgeCV(),
    'L1 CV Model': LassoCV(),
    'ElasticNetCV Model': ElasticNetCV()
}

In [220]:
x_test

,age,sex,bmi,smoker,ch_0,ch_1,ch_2,ch_3,ch_4,ch_5,re_northeast,re_northwest,re_southeast,re_southwest
559,-1.438764,0.989591,0.798339,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,1.765481,-0.611324,-0.566418
1087,1.266881,0.989591,0.143802,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,1.765481,-0.611324,-0.566418
1020,0.839674,0.989591,1.039485,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
460,0.697271,-1.010519,0.978788,-0.507463,-0.866781,-0.565267,-0.467525,2.742680,-0.137987,-0.116775,-0.565267,-0.566418,1.635795,-0.566418
802,-1.296362,0.989591,-1.371969,-0.507463,-0.866781,1.769076,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1.266881,0.989591,1.686640,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,1.769076,-0.566418,-0.611324,-0.566418
1268,-1.367563,-1.010519,0.432520,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,-0.611324,1.765481
134,-1.367563,-1.010519,-0.308141,-0.507463,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,1.769076,-0.566418,-0.611324,-0.566418
1274,-0.940356,0.989591,-0.591117,1.970587,1.153694,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.565267,-0.566418,1.635795,-0.566418


In [221]:
models

{'OLS Model': LinearRegression(),
 'L2 Model': Ridge(),
 'L1 Model': Lasso(),
 'ElasticNet Model': ElasticNet(),
 'L2 CV Model': RidgeCV(alphas=array([ 0.1,  1. , 10. ])),
 'L1 CV Model': LassoCV(),
 'ElasticNetCV Model': ElasticNetCV()}

In [223]:
#loopping through the models
for model in models.values():
    model.fit(x_train,y_train)
    print(model.score(x_test,y_test))
    

0.7380138769361969
0.7380257908998551
0.7380130758316127
0.661309269971605
0.7380257908998857
0.7379635391519065
0.1371255763787983
